In [39]:
import pandas as pd
from pathlib import Path
from gtin_validation import main

In [40]:
infoprice_df_path = Path.cwd() / "data" / "infoprice.csv"
products_df_path = output_file = Path.cwd() / "data" / "produtos.csv"

infoprice_df = pd.read_csv(infoprice_df_path, delimiter=",", encoding="utf-8")
products_df = pd.read_csv(products_df_path, delimiter=";", encoding="latin1")


C:\Users\marke\AppData\Local\Temp\ipykernel_8500\2403740105.py:5: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  products_df = pd.read_csv(products_df_path, delimiter=";", encoding="latin1")


In [41]:
products_df["Código de barras"] = products_df["Código de barras"].fillna(0).astype(str)
infoprice_df["ean"] = infoprice_df["ean"].fillna(0).astype("int64").astype(str)

In [42]:
products_df['Código de barras'] = products_df['Código de barras'].str.strip()
infoprice_df['ean'] = infoprice_df['ean'].str.strip()

In [43]:
print(products_df['Código de barras'].duplicated().sum())
print(infoprice_df['ean'].duplicated().sum())

360
44104056


In [44]:
infoprice_df = infoprice_df.drop_duplicates(subset='ean')
products_df = products_df.drop_duplicates(subset='Código de barras')

In [45]:
products_df["Válidos GTINs"] = products_df["Código de barras"].apply(main)
infoprice_df["valid"] = infoprice_df["ean"].apply(main)

In [46]:
products_df.describe(include="bool")

,Válidos GTINs
count,79428
unique,2
top,True
freq,75407


In [47]:
infoprice_df.describe(include="bool")

,valid
count,1791604
unique,2
top,True
freq,1748900


In [48]:
df = pd.merge(products_df, infoprice_df[['ean', 'valid']], left_on='Código de barras', right_on='ean', how='left')

In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79428 entries, 0 to 79427
Data columns (total 18 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   ID                        79428 non-null  int64 
 1   Ativo                     79428 non-null  int64 
 2   Código do produto         79428 non-null  int64 
 3   Nome do produto           79428 non-null  object
 4   Código de barras          79428 non-null  object
 5   Valor de venda            79428 non-null  object
 6   Valor de custo            79428 non-null  object
 7   NCM                       79428 non-null  int64 
 8   NCM (NOTAS)               79428 non-null  object
 9   Descontinuado             79428 non-null  int64 
 10  Categoria                 79428 non-null  object
 11  Fabricante                53311 non-null  object
 12  Fornecedor                1 non-null      object
 13  Unidade de medida         79428 non-null  object
 14  Data cadastro de produ

In [50]:
filtered_df = df[(df['Válidos GTINs'] == True) & (df['valid'] == True)]
filtered_products_df = products_df[(products_df['Válidos GTINs'] == True)]
filtered_infoprice_df = infoprice_df[(infoprice_df['valid'] == True)]

In [51]:
print(filtered_df.shape[0], filtered_products_df.shape[0], filtered_infoprice_df.shape[0])

54739 75407 1748900


In [52]:
output_path = Path.cwd() / "data" / "prod_validador_inforprice.xlsx"

df.to_excel(output_path, index=False)